In [1]:
import os
import fileinput

In [20]:
%%capture

#Merge areas into single COCO datasets for cross-validation
#=================================

def merge_areas(areas:list, tile_size):
    """
    Merge areas specified in areas to one coco file
    e.g merge areas = [1,2,3,4,5,6,7,8] etc.
    """
    if len(areas) <= 1:
        raise ValueError("Specify more than one area")

    merged_areas = str(areas[0]) + str(areas[1])

    #Merge first two areas without any removal
    os.system(f"python merge.py \
        almorox/tiled_{tile_size}/annotations/area_{areas[0]}_{tile_size}_COCO.json \
        almorox/tiled_{tile_size}/annotations/area_{areas[1]}_{tile_size}_COCO.json \
        almorox/tiled_{tile_size}/annotations/area_{merged_areas}_{tile_size}_COCO.json")


    for i in range(2, len(areas)): #Loop over r.h area
        print(areas[i])
        os.system(f"python merge.py \
        almorox/tiled_{tile_size}/annotations/area_{merged_areas}_{tile_size}_COCO.json \
        almorox/tiled_{tile_size}/annotations/area_{areas[i]}_{tile_size}_COCO.json \
        almorox/tiled_{tile_size}/annotations/area_{merged_areas + str(areas[i])}_{tile_size}_COCO.json")

        os.system(f"rm almorox/tiled_{tile_size}/annotations/area_{merged_areas}_{tile_size}_COCO.json")
        merged_areas += str(areas[i])

test_areas = list(range(1,10))
area_lists = [[x for x in range(1,10) if x != test_area] for test_area in test_areas]

tile_size = 1024

for areas in area_lists:
    merge_areas(areas, tile_size)

  0%|          | 0/205 [00:00<?, ?it/s]



Successfully merged the two files (almorox/tiled_1024/annotations/area_2_1024_COCO.json , almorox/tiled_1024/annotations/area_3_1024_COCO.json) into almorox/tiled_1024/annotations/area_23_1024_COCO.json


  0%|          | 0/265 [00:00<?, ?it/s]



Successfully merged the two files (almorox/tiled_1024/annotations/area_23_1024_COCO.json , almorox/tiled_1024/annotations/area_4_1024_COCO.json) into almorox/tiled_1024/annotations/area_234_1024_COCO.json


 92%|█████████▏| 244/265 [00:00<00:00, 6578.07it/s]


...



Successfully merged the two files (almorox/tiled_1024/annotations/area_12345_1024_COCO.json , almorox/tiled_1024/annotations/area_6_1024_COCO.json) into almorox/tiled_1024/annotations/area_123456_1024_COCO.json


 91%|█████████ | 424/467 [00:00<00:00, 4179.93it/s]




Successfully merged the two files (almorox/tiled_1024/annotations/area_123456_1024_COCO.json , almorox/tiled_1024/annotations/area_7_1024_COCO.json) into almorox/tiled_1024/annotations/area_1234567_1024_COCO.json


 90%|████████▉ | 487/544 [00:00<00:00, 3750.14it/s]




Successfully merged the two files (almorox/tiled_1024/annotations/area_1234567_1024_COCO.json , almorox/tiled_1024/annotations/area_8_1024_COCO.json) into almorox/tiled_1024/annotations/area_12345678_1024_COCO.json


In [24]:
#Generate cfg files for each area===========================================================
train_areas_str =  [''.join(map(str, area_list)) for area_list in area_lists]
test_areas_str = [str(test_area) for test_area in test_areas]
tile_size_str = str(tile_size)

base_cfg = "almorox/configs/BASE_almo_mask_rcnn_r101_fpn_mstrain-poly_3x_coco.py" 
out_dir = "almorox/configs/areas"

os.makedirs(out_dir, exist_ok=True)

with open(base_cfg, 'r') as base_file:
    base_data = base_file.read()

for train_areas, test_area in zip(train_areas_str, test_areas_str):
    data = base_data.replace("TSIZE", tile_size_str)
    data = data.replace("TRAINAREAS", train_areas)
    data = data.replace("TESTAREA", test_area)

    out_file = f"{out_dir}/TESTAREA{test_area}_almo_mask_rcnn_r101_fpn_mstrain-poly_3x_coco.py"
    with open(out_file, 'w') as out_file:
        out_file.write(data)


In [25]:
#Train all the models

config_dir = "almorox/configs/areas"

for test_area in ['1']:#test_areas_str:
    print("==================")
    print(f"TRAINING FOR TEST AREA {test_area}")
    print("==================")
    print("==================")
    print("==================")
    print("==================")
    print("==================")
    config_file = f"{config_dir}/TESTAREA{test_area}_almo_mask_rcnn_r101_fpn_mstrain-poly_3x_coco.py"
    os.system(f"python train.py {config_file}")

#!python ../train.py ../configs/mask_rcnn/almo_mask_rcnn_r101_fpn_mstrain-poly_3x_coco.py

TRAINING FOR TEST AREA 1


/home/matt/miniconda3/envs/mmdet/lib/python3.7/site-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '
/home/matt/miniconda3/envs/mmdet/lib/python3.7/site-packages/mmdet/utils/setup_env.py:39: UserWarning: Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  f'Setting OMP_NUM_THREADS environment variable for each process '
/home/matt/miniconda3/envs/mmdet/lib/python3.7/site-packages/mmdet/utils/setup_env.py:49: UserWarning: Setting MKL_NUM

loading annotations into memory...
Done (t=0.05s)
creating index...
index created!


2024-02-10 19:08:22,990 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2024-02-10 19:08:22,998 - mmdet - INFO - load checkpoint from local path: almorox/configs/mask_rcnn_r101_fpn_mstrain-poly_3x_coco_imagenet_checkpoint.pth


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


2024-02-10 19:08:23,197 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([2, 1024]).
size mismatch for roi_head.bbox_head.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([2]).
size mismatch for roi_head.bbox_head.fc_reg.weight: copying a param with shape torch.Size([320, 1024]) from checkpoint, the shape in current model is torch.Size([4, 1024]).
size mismatch for roi_head.bbox_head.fc_reg.bias: copying a param with shape torch.Size([320]) from checkpoint, the shape in current model is torch.Size([4]).
size mismatch for roi_head.mask_head.conv_logits.weight: copying a param with shape torch.Size([80, 256, 1, 1]) from checkpoint, the shape in current model is torch.Size([1, 256, 1, 1]).
size mismatch for roi_head.mask_head.conv_logits.bias

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 49/49, 5.8 task/s, elapsed: 8s, ETA:     0s

2024-02-10 19:13:16,491 - mmdet - INFO - Evaluating bbox...
2024-02-10 19:13:16,670 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.390
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.604
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.429
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.117
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.311
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.452
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.630
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.630
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.630
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.183
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.483
 Average R

Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.15s).
Accumulating evaluation results...
DONE (t=0.03s).
Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *segm*
DONE (t=0.20s).
Accumulating evaluation results...
DONE (t=0.03s).


2024-02-10 19:13:16,932 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.414
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.606
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.433
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.115
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.280
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.481
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.667
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.667
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.667
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.150
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.504
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000